In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import argparse
import torch

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import process_images, tokenizer_image_token, get_model_name_from_path

from PIL import Image
import torch.nn.functional as F

import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer
import numpy as np
import sys

IMAGE_PATH = "/code/yingqi/LLaVA_visiPruner/v1_68.png"

def load_image(image_file):
    if image_file.startswith('http://') or image_file.startswith('https://'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

if "ipykernel_launcher" in sys.argv[0]:
    sys.argv = sys.argv[:1]

parser = argparse.ArgumentParser()

parser.add_argument("--model-path", type=str, default="/code/yingqi/models/liuhaotian/llava-v1.5-7b")
parser.add_argument("--model-base", type=str, default=None)
parser.add_argument("--image-file", type=str, default=IMAGE_PATH)
parser.add_argument("--device", type=str, default="cuda")
parser.add_argument("--conv-mode", type=str, default="vicuna_v1")
parser.add_argument("--answer-with-sentence", type=bool, default=False)
parser.add_argument("--num-chunks", type=int, default=1)
parser.add_argument("--chunk-idx", type=int, default=0)
parser.add_argument("--temperature", type=float, default=0)
parser.add_argument("--top_p", type=float, default=None)
parser.add_argument("--num_beams", type=int, default=1)
parser.add_argument("--max_new_tokens", type=int, default=128)
parser.add_argument("--load-8bit", action="store_true")
parser.add_argument("--load-4bit", action="store_true")
parser.add_argument("--debug", action="store_true")
args = parser.parse_args()

disable_torch_init()

model_name = get_model_name_from_path(args.model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name, args.load_8bit, args.load_4bit, device=args.device)

if "llama-2" in model_name.lower():
    conv_mode = "llava_llama_2"
elif "mistral" in model_name.lower():
    conv_mode = "mistral_instruct"
elif "v1.6-34b" in model_name.lower():
    conv_mode = "chatml_direct"
elif "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"

lm_head = model.lm_head
norm = model.model.norm

/opt/conda/envs/llava-visipruner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/opt/conda/envs/llava-visipruner/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [02:56<00:00, 88.25s/it] 


In [6]:
if args.conv_mode is not None and conv_mode != args.conv_mode:
    print('[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}'.format(conv_mode, args.conv_mode, args.conv_mode))
else:
    args.conv_mode = conv_mode

conv = conv_templates[args.conv_mode].copy()
if "mpt" in model_name.lower():
    roles = ('user', 'assistant')
else:
    roles = conv.roles

image = load_image("/code/yingqi/LLaVA_visiPruner/images/v1_73.jpg")
image_size = image.size
image_tensor = process_images([image], image_processor, model.config)
if type(image_tensor) is list:
    image_tensor = [image.to(model.device, dtype=torch.float16) for image in image_tensor]
else:
    image_tensor = image_tensor.to(model.device, dtype=torch.float16)

inp = "How many cars?\nAnswer using a single word or a phrase."

print(f"{roles[1]}: ", end="")

if image is not None:
    # first message
    if model.config.mm_use_im_start_end:
        inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
    else:
        inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
    image = None

conv.append_message(conv.roles[0], inp)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

with torch.inference_mode():
    output = model.generate(
        input_ids,
        images=image_tensor.to(dtype=torch.float16, device='cuda', non_blocking=True),
        image_sizes=[image_size],
        do_sample=True if args.temperature > 0 else False,
        temperature=args.temperature,
        top_p=args.top_p,
        num_beams=args.num_beams,
        max_new_tokens=128,
        streamer=streamer,
        use_cache=True,
        output_hidden_states=True,
        return_dict_in_generate=True
    )



[WARNING] the auto inferred conversation mode is llava_v1, while `--conv-mode` is vicuna_v1, using vicuna_v1
ASSISTANT: 4


In [ ]:
# Please be noted: the hidden states of the last layer is recorded after the norm layer, while the states of previous layers are not.
for layer_idx, layer in enumerate(prefill_hidden_states):
    token_idx = -1
    logits = lm_head(norm(layer[0,token_idx,:])) if layer_idx != 31 else lm_head(layer[0,token_idx,:])
    probs, token_ids = logits.topk(5)
    decoded = [tokenizer.decode(token_id) for token_id in token_ids]
    print(f"Layer {layer_idx} top 5 tokens: {decoded}")

Layer 0 top tokens: ['Portail', 'ensoort', 'iemann', 'weis', '~[']
Layer 1 top tokens: ['Portail', 'archivi', 'embros', 'пута', 'sterd']
Layer 2 top tokens: ['sterd', 'Portail', 'Архив', 'férences', 'actér']
Layer 3 top tokens: ['sterd', 'férences', 'paździer', 'nahm', 'Архив']
Layer 4 top tokens: ['sterd', 'publique', 'paździer', 'Архив', 'Portail']
Layer 5 top tokens: ['sterd', 'paździer', 'strij', 'sierp', 'Außer']
Layer 6 top tokens: ['sterd', 'paździer', 'sierp', 'gior', 'Gemeins']
Layer 7 top tokens: ['sterd', 'typen', 'gior', 'othèque', 'dienst']
Layer 8 top tokens: ['penas', 'евич', 'intrag', '桥', 'kazy']
Layer 9 top tokens: ['sterd', 'shared', 'geldig', 'none', 'penas']
Layer 10 top tokens: ['number', 'zero', 'sterd', 'Zero', 'cyc']
Layer 11 top tokens: ['number', 'stor', 'VS', 'gem', 'cyc']
Layer 12 top tokens: ['gem', 'égl', 'zero', 'VS', 'stor']
Layer 13 top tokens: ['number', 'rita', 'zero', 'arden', 'none']
Layer 14 top tokens: ['number', 'arden', 'rita', 'ubre', 'nim']
L